# Perform decoding and encoding
Decode ROI and continuous images into text and encode text into continuous image.

In [1]:
# Import packages
from gclda.model import Model
from gclda.decode import Decoder

In [2]:
# Load example model and initialize decoder
model = Model.load('data/model.pkl')
decoder = Decoder(model)

In [4]:
# Encoding isn't working yet.
#text = 'painful stimulation during a language task'
#text_image = decoder.encode(text)

In [5]:
roi_file = 'data/fusiform.nii.gz'
fusiform_df = decoder.decode_roi(roi_file)
fusiform_df = fusiform_df.sort_values(by='Weight', ascending=False)
fusiform_df.head()

,Weight
Term,
visual,0.014622
motor,0.012857
attention,0.006233
auditory,0.006163
words,0.005935


In [7]:
continuous_file = 'data/faces_specificity_z.nii.gz'
emotion_df = decoder.decode_continuous(continuous_file)
emotion_df = emotion_df.sort_values(by='Weight', ascending=False)
emotion_df.head()

,Weight
Term,
visual,0.014817
motor,0.013414
attention,0.006351
auditory,0.005825
words,0.005761
